In [9]:
%cd "/home/y0065120/Dokumente/Leichtwerk/Projects/helics-opt/"

# Third-party imports
import pyansys
import pandas as pd
import numpy as np
from pyOpt import Optimization
from pyOpt import ALPSO
import time
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib as mpl
%matplotlib widget
import itkwidgets

# Local imports
# from femodel import Femodel
from util_mapdl import Material
from util_mapdl.post_functions import fc_puck


/home/y0065120/Dokumente/Leichtwerk/Projects/helics-opt


In [10]:
class Femodel:
    
    def __init__(self, mapdl, mesh_factor=20):
        self.mapdl = mapdl
        self.__ansys_input_filename = '_ansys_input_file'
        self.mesh_density_factor = mesh_factor

    def cdread(self):
        """
        Read in the .cdb database file generated by the pre-processing 
        method.

        """
        
        self.mapdl.prep7()
        self.mapdl.cdread('all', self.__ansys_input_filename, 'cdb')        
        
    def clear(self):
        """Resets the MAPDL Session. """
        
        self.mapdl.finish()
        self.mapdl.clear('NOSTART')
        
    def pre_processing(self):

        self.mapdl.prep7() # Enter Preprocessing Routine
        
        # Space for basic parameters and settings
        self.mapdl.seltol(1e-4)
        self.mapdl.et('1','SHELL281')    
        self.mapdl.keyopt(1,8,1)

        # Material Parameters
        for key in self.materials:
            self.materials[key].assign_mp()

        # Geometry settings
        b = 1000     # Breite 
        h = 1000     # Länge
        F = 100*1000     # Kraft

        self.mapdl.k(1,0,0,0)
        self.mapdl.k(2,b,0,0)
        self.mapdl.k(3,b,h,0)
        self.mapdl.k(4,0,h,0)
        self.mapdl.a(1,2,3,4)    
        
        self.mapdl.lsel('all')
        self.mapdl.lesize('all','','',self.mesh_density_factor)
        
        self.mapdl.mshkey(1)
        self.mapdl.mshape(0,'2d')
        self.mapdl.allsel('all')
        
        self.mapdl.sectype(100,'shell','','Dummy')
        self.mapdl.secdata(0.1,1,90.,3)
        self.mapdl.allsel('all')
        self.mapdl.amesh('all')     
        
        self.mapdl.nsel('s','loc','x',0)
        self.mapdl.nsel('r','loc','y',0)
        self.mapdl.d('all','all',0)

        self.mapdl.nsel('all')
        self.mapdl.d('all','uz',0)

        # Last
        self.mapdl.nsel('s','loc','x',b)
        self.mapdl.f('all','fy',F/self.mapdl.mesh.n_node)

        self.mapdl.nsel('s','loc','x',0)
        self.mapdl.f('all','fy',-F/self.mapdl.mesh.n_node)    

        self.mapdl.nsel('s','loc','y',h)
        self.mapdl.f('all','fx',F/self.mapdl.mesh.n_node)

        self.mapdl.nsel('s','loc','y',0)
        self.mapdl.f('all','fx',-F/self.mapdl.mesh.n_node)    

        self.mapdl.allsel('all')
        self.mapdl.cdwrite('all', self.__ansys_input_filename, 'cdb')
        self.mapdl.finish()
        self.mapdl.clear('nostart')
        
    def change_design_variables(self, alpha, beta, t_layer):

        self.mapdl.allsel('all')
        self.mapdl.sectype(1,'shell','','Secname')
        self.mapdl.secdata(t_layer,1,alpha,3)
        self.mapdl.secdata(t_layer,1,beta,3)
        self.mapdl.secdata(t_layer,1,beta,3)
        self.mapdl.secdata(t_layer,1,alpha,3)

        self.mapdl.emodif('all', 'secnum', 1)
        
    def solve(self):
        """Solve the FE-Model. """
        
        self.mapdl.run('/SOLU')
        self.mapdl.antype('static')
        self.mapdl.outres('all','all')
        self.mapdl.solve()
        self.mapdl.finish()
        
    def post_processing(self):
        
        self.mapdl.post1() # Enter Post-processing Routine
        
        # Assign Failure Criteria Values
        for key in self.materials:
            self.materials[key].assign_fc()
        
        self.mapdl.fctyp('dele','all')
        self.mapdl.fctyp('add','pmat')        
        
        self.mapdl.allsel('all')
        # mapdl.esel('s','cent','x',0.2,0.8)
        # mapdl.esel('r','cent','y',0.2,0.8)
        
        I_f, I_m = fc_puck(self.mapdl)
        
        return I_f, I_m
    
    def evaluate(self, x):
        self.cdread()
        self.change_design_variables(x[0], x[1], x[2])
        self.solve()
        _, I_m = self.post_processing()
        self.clear()
        
        return x[2], I_m
        
    @property
    def materials(self):
        """
        The Materials assigned to the Model.

        Returns
        -------
        materials : Dict

        """
        return self.__materials
    
    @materials.setter
    def materials(self, materials):
        self.__materials = materials
        
        for key in materials:
            self.__materials[key].load_from_db()
            
    @property
    def mesh_density_factor(self):
        """
        Integer factor to control the mesh density.

        Returns
        -------
        mesh_density_factor : int

        """
        return self.__mesh_density_factor
            
    @mesh_density_factor.setter
    def mesh_density_factor(self, mesh_density_factor):
        self.__mesh_density_factor = mesh_density_factor

In [11]:
ansys_path = '/home/y0065120/Dokumente/Leichtwerk/Projects/ansys-1/'
jobname ='job-0'

mapdl = pyansys.launch_mapdl(run_location=ansys_path,
                            nproc=1,
                            override=True,
                            loglevel='error',
                            additional_switches='-smp -d X11C',
                            jobname=jobname,
                            allow_ignore=True,
                            mode='console',
                            )

femodel = Femodel(mapdl)

femodel.materials = {'flaxpreg': Material(mapdl, 'FLAXPREG-T-UD', 1)}

femodel.pre_processing()

In [14]:
femodel.clear()
femodel.cdread()
femodel.change_design_variables(45,135,0.475)
femodel.solve()
I_f, I_m = femodel.post_processing()

print('I_m = ' + str(I_m))

I_m = 0.996934109


In [5]:
mapdl.fsum()

'FSUM,,\r\n\r\n ***** SUMMATION OF TOTAL    FORCES AND MOMENTS IN THE GLOBAL COORDINATE SYSTEM *****\r\n  FX  =  0.7980816E-10\r\n  FY  =  0.5138645E-10\r\n  FZ  =   0.000000    \r\n  MX  = -0.7516267E-24\r\n  MY  =  0.2475635E-24\r\n  MZ  = -0.1536591E-07\r\n\r\n SUMMATION POINT=  0.0000      0.0000      0.0000'

In [6]:
def objfunc(x):
    
    f, I_m = femodel.evaluate(x)
    
    objfunc.counter+= 1
    # print(str(objfunc.counter) + ' # ' + str(np.round(np.array(x),2)))        
    
    time.sleep(0.01)
    return f, list(np.array([I_m]) - 1), 0
objfunc.counter = 0

In [7]:
optprob = Optimization(name='Faserverbundplatte',
                        obj_fun=objfunc
                        )

# Add variables
optprob.addVar('alpha', 'c', lower=0., upper=90.)
optprob.addVar('beta', 'c', lower=90., upper=180.)
optprob.addVar('t_layer', 'c', lower=0.1,upper=1.)

# Add objective
optprob.addObj('f')

# Add constraints
optprob.addCon('g', 'i')

In [8]:
alpso_path = '/home/y0065120/Dokumente/Leichtwerk/Projects/ALPSO/'
filename = 'Faserverbundplatte_Output_ALPSO'

alpso = ALPSO()
alpso.setOption('fileout',1)
alpso.setOption('filename', alpso_path+filename)

alpso.setOption('SwarmSize',40)
alpso.setOption('stopIters',2)

def coldstart():    
    alpso(optprob, store_hst=True)
    print(optprob.solution(0))